In [1]:
# Add PATH (adapt!)
push!(LOAD_PATH, "/Volumes/cfranken/code/gitHub/LSM-SPAM/src/Leaf/");
push!(LOAD_PATH, "/Volumes/cfranken/code/gitHub/LSM-SPAM/src/Utils/");
#push!(LOAD_PATH, "/Volumes/cfranken/code/gitHub/LSM-SPAM/src/tools/");

# Use Plots:
using Plots
#using PhotoStructs
plotly()

┌ Info: For saving to png with the Plotly backend ORCA has to be installed.
└ @ Plots /Users/cfranken/.julia/packages/Plots/NVH6y/src/backends.jl:363


Plots.PlotlyBackend()

In [2]:
using CanopyRTMod
using BenchmarkTools
using Statistics

┌ Info: Precompiling CanopyRTMod [top-level]
└ @ Base loading.jl:1260


Reading Optical Parameters from 400.0 to 2380.0 length: 115


In [3]:
wl = CanopyRTMod.wl;
wle = CanopyRTMod.wle;
wlf = CanopyRTMod.wlf;

In [4]:
arrayOfLeaves = Array{leafbio{FT}, 1}(undef, CanopyRTMod.canopy.nlayers)
for i = 1:CanopyRTMod.canopy.nlayers
    arrayOfLeaves[i] = leafbio{FT, length(wl), length(wle), length(wlf)}()
    CanopyRTMod.fluspect!(arrayOfLeaves[i], CanopyRTMod.optis)
end

In [45]:
reflVIS = []
reflNIR = []
for VZA=-85:1:85
    CanopyRTMod.angles.tto=VZA
    CanopyRTMod.computeCanopyGeomProps!(canopy, angles,canOpt);
    CanopyRTMod.computeCanopyMatrices!(arrayOfLeaves,canOpt);
    CanopyRTMod.RTM_SW!(canopy, canOpt, canRad,sunRad, CanopyRTMod.soil);
    push!(reflVIS, canRad.alb_obs[28])
    push!(reflNIR, canRad.alb_obs[52])
end

In [46]:
@show wl[28]
@show wl[52]

wl[28] = 662.0f0
wl[52] = 817.0f0


817.0f0

In [47]:
VZA=-85:1:85
plot(VZA, reflVIS)

<!DOCTYPE html>
 
 
 Plots.jl

In [39]:
VZA=-85:1:85
plot(VZA, reflNIR)

<!DOCTYPE html>
 
 
 Plots.jl

In [82]:
reflRed_SZA = []
reflNIR_SZA = []
CanopyRTMod.angles.tto=0
CanopyRTMod.canopy.LAI=4.
for SZA=0.0:1:75
    CanopyRTMod.angles.tts=SZA
    CanopyRTMod.computeCanopyGeomProps!(canopy, angles,canOpt);
    CanopyRTMod.computeCanopyMatrices!(arrayOfLeaves,canOpt);
    CanopyRTMod.RTM_SW!(canopy, canOpt, canRad,sunRad, CanopyRTMod.soil);
    push!(reflRed_SZA, canRad.alb_direct[28])
    push!(reflNIR_SZA, canRad.alb_direct[52])
end

In [83]:
SZA=0:1:75
plot(SZA, reflNIR_SZA)

<!DOCTYPE html>
 
 
 Plots.jl

In [84]:
SZA=0:1:75
plot(SZA, reflRed_SZA)

<!DOCTYPE html>
 
 
 Plots.jl

114-element view(::Array{Float32,1}, :) with eltype Float32:
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 ⋮
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1